In [1]:
!apt-get install -y hmmer prodigal
!pip install tqdm pandas openpyxl

from google.colab import drive, files
drive.mount('/content/drive')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdivsufsort3
Suggested packages:
  hmmer-doc
The following NEW packages will be installed:
  hmmer libdivsufsort3 prodigal
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,838 kB of archives.
After this operation, 19.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdivsufsort3 amd64 2.0.1-5 [42.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 prodigal amd64 1:2.6.3-5 [640 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 hmmer amd64 3.3.2+dfsg-1 [1,155 kB]
Fetched 1,838 kB in 1s (1,788 kB/s)
Selecting previously unselected package libdivsufsort3:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../libdivsufsort3_2.0.1-5_amd64.deb ...
Unpacking libdivsufsort3:amd64 (2.0

In [7]:
import os
import subprocess
from pathlib import Path

fna_dir = Path("/content/drive/MyDrive/genome_data/fna_output")
faa_dir = Path("/content/drive/MyDrive/genome_data/faa_output")
hmmer_dir = Path("/content/hmmscan_outputs")
pfam_db = '/content/drive/MyDrive/genome_data/Pfam-A.hmm'

faa_dir.mkdir(parents=True, exist_ok=True)
hmmer_dir.mkdir(parents=True, exist_ok=True)

In [4]:
import shutil

for fna_file in fna_dir.glob("*.fna"):
  base_name = fna_file.stem
  faa_out = faa_dir / f"{base_name}.faa"

  cmd = f"prodigal -i {fna_file} -a {faa_out} -p meta -q"
  subprocess.run(cmd, shell=True, check=True)

  shutil.copy(faa_out , Path("/content/drive/MyDrive/genome_data/faa_output") / f"{base_name}.faa")
  print(f"Converted {fna_file.name} to {faa_out.name}")

print("All .fna files translated to .faa")

✅ Converted 751_GCF_000017885.4_ASM1788v4_genomic.fna to 751_GCF_000017885.4_ASM1788v4_genomic.faa
✅ Converted 698_GCF_000008425.1_ASM842v1_genomic.fna to 698_GCF_000008425.1_ASM842v1_genomic.faa
✅ All .fna files translated to .faa


In [ ]:
import subprocess

hmmer_dir.mkdir(parents=True, exist_ok=True)

for faa_file in faa_dir.glob("*.faa"):
  if not faa_file.exists():
    print(f"[!] Skipping missing file: {faa_file}")
    continue
  if faa_file.stat().st_size == 0:
    print(f"[!] Skipping empty file: {faa_file.name}")
    continue

  out_txt = hmmer_dir / f"{faa_file.stem}.hmmscan.tbl"

  try:
    subprocess.run(
      ["hmmscan", "--tblout", str(out_txt), str(pfam_db), str(faa_file)],
      shell=False,
      check=True,
      capture_output=True
    )
    print(f"Processed {faa_file.name}")
  except subprocess.CalledProcessError as e:
    print(f"[!] Error running hmmscan on {faa_file.name}: {e}")
    print(e.stderr)
  except FileNotFoundError:
    print("[!] hmmscan not found. Make sure HMMER is installed and in PATH.")

print("All hmmscan jobs attempted")

In [6]:
import pandas as pd

results = []
for out_file in hmmer_dir.glob("*.tbl"):
  with open(out_file) as f:
    for line in f:
      if line.startswith("#"): continue
      parts = line.strip().split()
      if len(parts) >= 18:
        results.append({
          "Strain": out_file.stem.split('_')[0],
          "Pfam": parts[1].split('.')[0],
          "E-value": float(parts[4]),
          "Score": float(parts[5]),
          "Target": parts[2]
        })

df = pd.DataFrame(results)
df.to_excel("/content/drive/MyDrive/genome_data/pfam_annotations.xlsx", index=True)
print("Saved: pfam_annotations.xlsx")

📄 Saved: pfam_annotations.xlsx


In [9]:
def count_pfam_hits(outfile):
  with open(outfile) as f:
    lines = [line.strip() for line in f if not line.startswith('#') and line.strip()]

  pfam_ids = []
  for line in lines:
    parts = line.split()
    if len(parts) > 1:
      pfam_ids.append(parts[1].split('.')[0])

  counts = pd.Series(pfam_ids).value_counts().reset_index()
  counts.columns = ["Pfam_ID", "Hit_Count"]
  return counts

tblout = hmmer_dir.glob("*.tbl")

for f in tblout:
  pfam_counts_df = count_pfam_hits(f)
  pfam_counts_df.to_excel("pfam_hit_counts.xlsx", index=False)
  files.download("pfam_hit_counts.xlsx")

pfam_counts_df.head(10)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Pfam_ID,Hit_Count
0,PF13401,143
1,PF00005,127
2,PF13191,117
3,PF13412,105
4,PF13555,102
5,PF08279,100
6,PF03193,97
7,PF02463,94
8,PF13384,90
9,PF00004,90
